## Decision Trees

Import the libraries

In [1]:
import wget
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

Imagine that you are a medical researcher compiling data for a study. You have collected data about a set of patients, all of whom suffered from the same illness. During their course of treatment, each patient responded to one of **5 medications, Drug A, Drug B, Drug c, Drug x and y.**

Part of your job is to build a model to find out which drug might be appropriate for a future patient with the same illness. **The feature sets of this dataset are Age, Sex, Blood Pressure, and Cholesterol of patients, and the target is the drug that each patient responded to.**

It is a sample of binary classifier, and you can use the training part of the dataset to build a decision tree, and then use it to predict the class of a unknown patient, or to prescribe it to a new patient.

Download the data:

In [2]:
drug200 = wget.download("https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/drug200.csv")

100% [................................................................................] 6027 / 6027

Read data using pandas dataframe:

In [3]:
my_data = pd.read_csv(drug200, delimiter=",")
my_data[0:5]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


Size of data

In [4]:
my_data.shape

(200, 6)

### Pre-processing

Using `my_data` as the Drug.csv data read by pandas, declare the following variables:

- X as the Feature Matrix (data of my_data)
- y as the response vector (target)

In [5]:
# Remove the column containing the target name since it doesn't contain numeric values
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.113999999999999],
       [28, 'F', 'NORMAL', 'HIGH', 7.797999999999999],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

As you may figure out, some features in this dataset are categorical such as Sex or BP. Unfortunately, Sklearn Decision Trees do not handle categorical variables. But still we can convert these features to numerical values. pandas.get_dummies() Convert categorical variable into dummy/indicator variables.

In [6]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]

array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.113999999999999],
       [28, 0, 2, 0, 7.797999999999999],
       [61, 0, 1, 0, 18.043]], dtype=object)

Now we can fill the target variable

In [7]:
y = my_data["Drug"]
y[0:5]

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

#### Setting up the Decision Tree

We will be using train/test split on our decision tree. Let's import `train_test_split` from `sklearn.cross_validation`.

In [8]:
from sklearn.model_selection import train_test_split

Now `train_test_split` will return 4 different parameters. We will name them:
`X_trainset, X_testset, y_trainset, y_testset`

The `train_test_split` will need the parameters:
`X, y, test_size=0.3, and random_state=3`.

The X and y are the arrays required before the split, the test_size represents the ratio of the testing dataset, and the `random_state` ensures that we obtain the same splits.

In [9]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)
print(X_trainset.shape)
print(X_testset.shape)
print(y_trainset.shape)
print(y_testset.shape)

(140, 5)
(60, 5)
(140,)
(60,)


#### Modelling

We will first create an instance of the `DecisionTreeClassifier` called `drugTree`.

Inside of the classifier, specify `criterion="entropy"` so we can see the information gain of each node.

Next, we will fit the data with the training feature matrix `X_trainset` and training response vector `y_trainset`.

In [10]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

#### Prediction

Let's make some predictions on the testing dataset and store it into a variable called `predTree`.

In [11]:
predTree = drugTree.predict(X_testset)
print (predTree [0:5])
print (y_testset [0:5])

['drugY' 'drugX' 'drugX' 'drugX' 'drugX']
40     drugY
51     drugX
139    drugX
197    drugX
170    drugX
Name: Drug, dtype: object


#### Evaluation

In [12]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  0.9833333333333333
